In [1]:
import numpy as np
%matplotlib inline
%matplotlib ipympl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.animation import FuncAnimation
import pandas as pd
import math
import tensorflow as tf
from lib.pinn import PINN
from lib.network import Network
import pathlib

In [2]:
def train(pinn, p, epochs=100):
    n_train_samples = 2

    max_d = np.array([p["max_t"], p["max_x"], p["max_y"], p["max_z"]])

    x_train = np.random.rand(n_train_samples, 4) * max_d
    x_train_left = np.random.rand(n_train_samples, 4) * max_d
    x_train_left[:,1] = 0
    x_train_right = np.random.rand(n_train_samples, 4) * max_d
    x_train_right[:,1] = 0
    x=[x_train, x_train_left, x_train_right]

    y = [np.zeros((n_train_samples, 1))] * 11
    
    pinn.fit(x=x, y=y, epochs=epochs)
    # pinn.save_weights('weights.h5')


In [3]:
# number of test samples
num_test_samples = 10

# system parameters
p: dict[str, float] = {
    "D": 0.006075,
    "W": 0.,
    "q": 1.,
    "l": 0.7095,
    "kT": 1.,
    "eta": 79.53,
    "m": 1.,
    "Fi_left": 1,
    "Fi_right": 1.,
    "n_left": 1.,
    "n_right": 1.,
    "width": 50.,
    "max_t": 100.,
    "max_x": 100.,
    "max_y": 6.,
    "max_z": 6.,
    "density_water": 26.15,
}
p["nu"] = p["D"]/p["kT"]
p["eta_b"] = p["eta"]/p["density_water"]

# build a core network model
network = Network.build()
# network.summary()

In [4]:
# build a PINN model
pinn = PINN(network, p["m"]).build(**p)
pinn.compile(optimizer='adam', loss='mse')
if pathlib.Path("weights.h5").exists():
    pinn.load_weights("weights.h5")

In [5]:
if False:
    train(pinn, p)
    pinn.save_weights("weights.h5")
    

Epoch 1/600
1/1 [==============================] - 14s 14s/step - loss: 74.6779 - tf.__operators__.add_loss: 9.5225e-05 - tf.math.subtract_loss: 8.7391e-10 - tf.__operators__.add_3_loss: 62.5294 - tf.__operators__.add_7_loss: 0.1685 - tf.__operators__.add_8_loss: 9.5225e-05 - tf.math.subtract_2_loss: 1.9439 - gradient_layer_loss: 0.5719 - tf.math.subtract_3_loss: 5.3490 - tf.math.subtract_4_loss: 0.4250 - gradient_layer_1_loss: 0.5493 - tf.math.subtract_5_loss: 3.1406
Epoch 2/600
1/1 [==============================] - 0s 8ms/step - loss: 60.7321 - tf.__operators__.add_loss: 9.0119e-05 - tf.math.subtract_loss: 6.3063e-10 - tf.__operators__.add_3_loss: 49.1300 - tf.__operators__.add_7_loss: 0.1734 - tf.__operators__.add_8_loss: 9.0119e-05 - tf.math.subtract_2_loss: 1.7450 - gradient_layer_loss: 0.5437 - tf.math.subtract_3_loss: 5.2249 - tf.math.subtract_4_loss: 0.3208 - gradient_layer_1_loss: 0.5280 - tf.math.subtract_5_loss: 3.0661
Epoch 3/600
1/1 [==============================] - 0s 7

In [6]:
num_test_samples = 30
t_flat = np.linspace(0, p["max_t"], num_test_samples)
x_flat = np.linspace(0, p["max_x"], num_test_samples)
y_flat = np.linspace(0, p["max_y"], num_test_samples)
z_flat = np.linspace(0, p["max_z"], num_test_samples)
t, x, y, z = np.meshgrid(t_flat, x_flat, y_flat, z_flat)
txyz = np.stack([t.flatten(), x.flatten(),
                y.flatten(), z.flatten()], axis=-1)

n, v, Fi = network.predict(txyz, batch_size=num_test_samples)

27000/27000 [==============================] - 16s 572us/step


In [ ]:
n = n.reshape(t.shape)
time_t = 0
fig = plt.figure(figsize=(16, 4.5))
ax1 = fig.add_subplot(131)
line, = ax1.plot(x_flat, n[0, :, int(p["max_y"]/2), int(p["max_z"]/2)])
ax2 = fig.add_subplot(132)
ax2.quiver(x, y, v_xy[..., 0], v_xy[..., 1])
ax3 = fig.add_subplot(133)
ax3.plot(x_flat, v[time_t, :, int(p["max_y"]/2), int(p["max_z"]/2), 0])


plt.show()


In [1]:
n_train_samples = 1
max_d = np.array([p["max_t"], p["max_x"], p["max_y"], p["max_z"]])
x_train = np.random.rand(n_train_samples, 4) * max_d
x_train_left = np.random.rand(n_train_samples, 4) * max_d
x_train_left[:,1] = 0
x_train_right = np.random.rand(n_train_samples, 4) * max_d
x_train_right[:,1] = 0
x=[x_train, x_train_left, x_train_right]
pinn(x)


NameError: name 'np' is not defined

In [ ]:

def anim(time_t):
    global v
    # ax1.plot(x_flat, n[time_t, :, int(p["max_y"]/2), int(p["max_z"]/2)])
    line.set_ydata(n[time_t, :, int(p["max_y"]/2), int(p["max_z"]/2)])
    x, y = np.meshgrid(x_flat, y_flat)
    v = v.reshape(t.shape + (3,))
    v_xy = v[time_t, :, :, int(p["max_z"]/2), 0:2]
    ax2.quiver(x, y, v_xy[..., 0], v_xy[..., 1])

    ax3.plot(x_flat, v[time_t, :, int(p["max_y"]/2), int(p["max_z"]/2), 0])

# animation = FuncAnimation(fig, anim, frames=int(p["max_t"]),)